<a href="https://colab.research.google.com/github/annechris13/Master-Thesis/blob/master/performance_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import make_spd_matrix
import random
import warnings
warnings.filterwarnings('error')
import time
import cvxpy as cp
from scipy.linalg import sqrtm

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install import-ipynb
import import_ipynb

In [4]:
%cd "/content/gdrive/My Drive/Colab Notebooks"
import mpc_class
import qpth_class

/content/gdrive/My Drive/Colab Notebooks
importing Jupyter notebook from mpc_class.ipynb
importing Jupyter notebook from qpth_class.ipynb


In [0]:
#initialize the mpc solver class 
mpc_solver=mpc_class.mpc()

In [6]:
%cd "/content"
df=pd.read_csv('gdrive/My Drive/test_cases.csv')
seeds=df["seed"].astype(int)
# df.head()

/content


In [0]:
def qp_cvxpy(n,P,q,G,h,A,b):
    x= cp.Variable(n)
    objective=cp.Minimize((1/2)*cp.sum_squares(P*x) + q.T @ x)
    constraints=[G@x <=h,A@x ==b]
    prob=cp.Problem(objective,constraints)
    prob.solve(verbose=False)
    cvxpy_optimum= prob.value
    cvxpy_solution=x.value
    return cvxpy_optimum,cvxpy_solution

# Test on Batch of problems

In [0]:
import random

In [0]:
ip="csv"
df=df.iloc[:,:8]
nbatch=len(df)
nx=6
nineq=6
neq=3
Q=[]
p=[]
G=[]
h=[]
A=[]
b=[]
for i in range(nbatch):
  #generate random problems using seeds in the csv file
  seed=int(df["seed"][i])
  random.seed(seed)
  Q.append(make_spd_matrix(nx,random_state=seed).reshape(1,-1))
  p.append([random.randint(0,9) for i in range(nx)])#.astype(float))
  G.append([random.randint(0,9)*((-1)**random.randint(0,1)) for i in range(nineq*nx)])#.astype(float))
  h.append([0 for i in range(nineq)])#.astype(float))
  A.append([random.randint(0,9) for i in range(neq*nx)])#.astype(float))
  b.append([random.randint(0,9) for i in range(neq)])#.astype(float)  )

Q=torch.tensor(Q).view(nbatch,nx,nx).type(torch.DoubleTensor)
p=torch.tensor(p).view(nbatch,nx).type(torch.DoubleTensor)
G=torch.tensor(G).view(nbatch,nineq,nx).type(torch.DoubleTensor)
h=torch.tensor(h).view(nbatch,nineq).type(torch.DoubleTensor)
A=torch.tensor(A).view(nbatch,neq,nx).type(torch.DoubleTensor)
b=torch.tensor(b).view(nbatch,neq).type(torch.DoubleTensor)

Q_batched=Q
p_batched=p

In [10]:
#solve using mpc solver
start=time.time()
x_mpc,_=mpc_solver.solve(Q,p,G,h,A,b)
mpc_time=time.time()-start
#solve using qpt solver
start=time.time()
x_qpt,_,_,_=qpth_class.qpth_opt(Q,p,G,h,A,b)
qpth_time=time.time()-start
#print run times
print("mpc time: ", mpc_time)
print("qpth time: ", qpth_time)

no of mu not converged:  12
mpc warning: Residuals not converged, need more itrations
qpth warning: Returning an inaccurate and potentially incorrect solution.
mpc time:  29.887669801712036
qpth time:  29.180514097213745


In [11]:
cols_mpc=[str(i)+"mpc" for i in range(nx)]
cols_qpt=[str(i)+"qpt" for i in range(nx)]
cols_cp=[str(i)+"cp" for i in range(nx)]
error_qpt=np.zeros(nbatch)
error_mpc=np.zeros(nbatch)
#results of mpc and qpt as data frames
new_mpc=pd.DataFrame(x_mpc.numpy().round(6).reshape(nbatch,-1),columns=cols_mpc)
new_qpt=pd.DataFrame(x_qpt.numpy().round(6).reshape(nbatch,-1),columns=cols_qpt)
#results of cvxpy from the csv file
tmp=df.copy()
for i in range(nx):
    col_mpc=cols_mpc[i]
    col_qpt=cols_qpt[i]
    #copy mpc and qpt results to the tmp df
    tmp[col_mpc]=new_mpc[col_mpc].copy()
    tmp[col_qpt]=new_qpt[col_qpt].copy()
    #calculate difference of optimal points from cvxpy (elementwise)
    error_mpc+=abs(np.round(tmp[str(i)+"cp"],6)-np.round(tmp[col_mpc],6))
    error_qpt+=abs(np.round(tmp[str(i)+"cp"],6)-np.round(tmp[col_qpt],6))
#calculate optimum values and store them
cp_=torch.tensor(np.array(tmp[cols_cp])).view(nbatch,nx,1)
cp_opt=np.round(((0.5*torch.bmm(torch.transpose(cp_,dim0=2,dim1=1),
                                torch.bmm(Q,cp_)))+torch.bmm(p.unsqueeze(-1).transpose(2,1),cp_)).view(-1).numpy(),6)
tmp["cp-opt"]=cp_opt
mpc=torch.tensor(np.array(tmp[cols_mpc])).view(nbatch,nx,1)
mpc_opt=np.round(((0.5*torch.bmm(torch.transpose(mpc,dim0=2,dim1=1),
                                 torch.bmm(Q,mpc)))+torch.bmm(p.unsqueeze(-1).transpose(2,1),mpc)).view(-1).numpy(),6)
tmp["mpc-opt"]=mpc_opt
qpt=torch.tensor(np.array(tmp[cols_qpt])).view(nbatch,nx,1)
qpt_opt=np.round(((0.5*torch.bmm(torch.transpose(qpt,dim0=2,dim1=1),
                                 torch.bmm(Q,qpt)))+torch.bmm(p.unsqueeze(-1).transpose(2,1),qpt)).view(-1).numpy(),6)
tmp["qpt-opt"]=qpt_opt

#store error values calculated in the for loop
tmp["cp-mpc-error"]=np.round(error_mpc,6)
tmp["cp-mpc-er_flag"]=np.round(error_mpc,6)!=0
tmp["cp-qpt-error"]=np.round(error_qpt,6)
tmp["cp-qpt-er_flag"]=np.round(error_qpt,6)!=0


print("No.of problems solved: ",len(tmp))
cvxpy_list=[]
for i in range(len(tmp)-1): #remove rows where cvxpy threw error
    if all(tmp[cols_cp].iloc[i,:]==0):
      cvxpy_list.append(i)
tmp=tmp.drop(cvxpy_list,axis=0)
print("No.of errors from MPC Method: ",len(tmp[tmp['cp-mpc-error']!=0])," (",round(len(tmp[tmp['cp-mpc-error']!=0])*100/len(tmp),2),"%)")
print("No.of errors from qpth: ",len(tmp[tmp['cp-qpt-error']!=0])," (",round(len(tmp[tmp['cp-qpt-error']!=0])*100/len(tmp),2),"%)")
cols_order=["seed","cp-opt","mpc-opt","qpt-opt","cp-mpc-error","cp-mpc-er_flag","cp-qpt-error","cp-qpt-er_flag"]+ cols_cp+cols_mpc+cols_qpt
tmp=tmp[cols_order]

No.of problems solved:  10000
No.of errors from MPC Method:  26  ( 0.26 %)
No.of errors from qpth:  28  ( 0.28 %)


In [12]:
all(tmp[cols_cp].iloc[131,:]==0)
# tmp=tmp.drop(131,axis=0)
# for i in range(len(tmp)):
#   print(tmp[cols_cp].iloc[i,:])
len(tmp)

9984

In [13]:
tmp[tmp["cp-mpc-er_flag"]]

,seed,cp-opt,mpc-opt,qpt-opt,cp-mpc-error,cp-mpc-er_flag,cp-qpt-error,cp-qpt-er_flag,0cp,1cp,2cp,3cp,4cp,5cp,0mpc,1mpc,2mpc,3mpc,4mpc,5mpc,0qpt,1qpt,2qpt,3qpt,4qpt,5qpt
1303,3251.0,1079.911924,1075.866849,1075.866849,0.307989,True,0.307989,True,39.207787,-39.756540,9.266674,-24.515612,31.604835,14.898972,39.132071,-39.679418,9.249718,-24.468080,31.542957,14.870187,39.132071,-39.679418,9.249718,-24.468080,31.542957,14.870187
1499,3709.0,2953.490568,2953.490548,2953.490548,0.000001,True,0.000001,True,-20.992844,17.648964,-29.570810,23.216572,-0.870245,0.738983,-20.992844,17.648964,-29.570810,23.216573,-0.870245,0.738983,-20.992844,17.648964,-29.570810,23.216573,-0.870245,0.738983
1563,3885.0,5.826862,5.910509,5.826862,0.150412,True,0.000000,False,0.211054,0.100187,0.419184,0.417974,0.302179,-0.123173,0.230354,0.082182,0.396667,0.442659,0.324449,-0.166808,0.211054,0.100187,0.419184,0.417974,0.302179,-0.123173
1654,4085.0,104.411748,104.076363,104.076363,0.042875,True,0.042875,True,-2.387754,1.054420,-0.800544,1.278809,9.136928,-3.278140,-2.381643,1.048309,-0.798309,1.277778,9.115942,-3.271739,-2.381643,1.048309,-0.798309,1.277778,9.115942,-3.271739
2090,5130.0,3693.790481,3658.645187,3658.645187,0.402371,True,0.402371,True,-20.239846,19.914573,-5.128942,22.540444,-9.099782,5.446464,-20.141635,19.815589,-5.098859,22.431559,-9.057034,5.423004,-20.141635,19.815589,-5.098859,22.431559,-9.057034,5.423004
2795,6815.0,128.800444,127.902069,127.902069,0.075568,True,0.075568,True,-2.736962,0.879507,-3.153703,1.394740,-5.356858,6.943179,-2.725664,0.876106,-3.141593,1.389381,-5.336283,6.920354,-2.725664,0.876106,-3.141593,1.389381,-5.336283,6.920354
2842,6916.0,11631.000912,11294.337377,11294.337377,5.177131,True,5.177131,True,-1.357289,-45.782718,43.433463,66.969304,-167.384097,-20.964545,-1.333333,-45.071429,42.785714,65.976190,-164.904762,-20.642857,-1.333333,-45.071429,42.785714,65.976190,-164.904762,-20.642857
3477,8508.0,159.332401,160.008859,160.008859,0.112799,True,0.112799,True,-15.379678,5.795875,21.712877,7.981493,0.111556,-12.831594,-15.407703,5.805959,21.750727,7.994913,0.111919,-12.854651,-15.407703,5.805959,21.750727,7.994913,0.111919,-12.854651
4809,11761.0,166050.666640,166051.104405,166051.104405,0.001871,True,0.001871,True,-485.072165,227.559093,-82.505642,-211.118501,407.049279,-28.501878,-485.072797,227.559387,-82.505747,-211.118774,407.049808,-28.501916,-485.072797,227.559387,-82.505747,-211.118774,407.049808,-28.501916
5275,12879.0,177623.126784,177624.312304,177624.312304,0.004494,True,0.004494,True,685.634070,-193.201697,78.967478,-93.108194,195.459756,-96.084722,685.636364,-193.202346,78.967742,-93.108504,195.460411,-96.085044,685.636364,-193.202346,78.967742,-93.108504,195.460411,-96.085044


In [14]:
tmp[tmp["cp-qpt-er_flag"]]

,seed,cp-opt,mpc-opt,qpt-opt,cp-mpc-error,cp-mpc-er_flag,cp-qpt-error,cp-qpt-er_flag,0cp,1cp,2cp,3cp,4cp,5cp,0mpc,1mpc,2mpc,3mpc,4mpc,5mpc,0qpt,1qpt,2qpt,3qpt,4qpt,5qpt
1303,3251.0,1079.911924,1075.866849,1075.866849,0.307989,True,0.307989,True,39.207787,-39.756540,9.266674,-24.515612,31.604835,14.898972,39.132071,-39.679418,9.249718,-24.468080,31.542957,14.870187,39.132071,-39.679418,9.249718,-24.468080,31.542957,14.870187
1499,3709.0,2953.490568,2953.490548,2953.490548,0.000001,True,0.000001,True,-20.992844,17.648964,-29.570810,23.216572,-0.870245,0.738983,-20.992844,17.648964,-29.570810,23.216573,-0.870245,0.738983,-20.992844,17.648964,-29.570810,23.216573,-0.870245,0.738983
1654,4085.0,104.411748,104.076363,104.076363,0.042875,True,0.042875,True,-2.387754,1.054420,-0.800544,1.278809,9.136928,-3.278140,-2.381643,1.048309,-0.798309,1.277778,9.115942,-3.271739,-2.381643,1.048309,-0.798309,1.277778,9.115942,-3.271739
2090,5130.0,3693.790481,3658.645187,3658.645187,0.402371,True,0.402371,True,-20.239846,19.914573,-5.128942,22.540444,-9.099782,5.446464,-20.141635,19.815589,-5.098859,22.431559,-9.057034,5.423004,-20.141635,19.815589,-5.098859,22.431559,-9.057034,5.423004
2417,5910.0,14.416146,14.416146,14.445661,0.000000,False,0.090333,True,0.034153,0.340292,0.233124,0.122422,-0.695894,1.752989,0.034153,0.340292,0.233124,0.122422,-0.695894,1.752989,0.050554,0.325828,0.265256,0.132445,-0.709966,1.756230
2453,5994.0,-2.716640,-2.716640,-2.707850,0.000000,False,0.475329,True,-0.398802,0.692751,-3.456367,2.864062,-0.157545,-1.663184,-0.398802,0.692751,-3.456367,2.864062,-0.157545,-1.663184,-0.359470,0.755288,-3.291154,2.829568,-0.129539,-1.808931
2795,6815.0,128.800444,127.902069,127.902069,0.075568,True,0.075568,True,-2.736962,0.879507,-3.153703,1.394740,-5.356858,6.943179,-2.725664,0.876106,-3.141593,1.389381,-5.336283,6.920354,-2.725664,0.876106,-3.141593,1.389381,-5.336283,6.920354
2842,6916.0,11631.000912,11294.337377,11294.337377,5.177131,True,5.177131,True,-1.357289,-45.782718,43.433463,66.969304,-167.384097,-20.964545,-1.333333,-45.071429,42.785714,65.976190,-164.904762,-20.642857,-1.333333,-45.071429,42.785714,65.976190,-164.904762,-20.642857
3477,8508.0,159.332401,160.008859,160.008859,0.112799,True,0.112799,True,-15.379678,5.795875,21.712877,7.981493,0.111556,-12.831594,-15.407703,5.805959,21.750727,7.994913,0.111919,-12.854651,-15.407703,5.805959,21.750727,7.994913,0.111919,-12.854651
3498,8563.0,14.324313,14.324313,14.329567,0.000000,False,0.151873,True,0.409810,0.941327,-2.443849,-0.206999,1.982945,0.211897,0.409810,0.941327,-2.443849,-0.206999,1.982945,0.211897,0.430370,0.880435,-2.439221,-0.161959,1.984091,0.231504


# Test on individual problems

In [0]:
n=6
m=6
p=3
nbatch=1

In [16]:
import random as r
cvxpy_time=0.0
mpc_time=0.0
qpth_time=0.0
columns=["seed"]+[str(i)+"cp" for i in range(n)]
results=pd.DataFrame([],columns=columns)
for k,s in enumerate(seeds):
    r.seed(s)
    P=make_spd_matrix(n, random_state=s)
    q=np.array([r.randint(0,9) for i in range(n)]).reshape(n).astype(float)
    G=np.array([r.randint(0,9)*((-1)**r.randint(0,1)) for i in range(m*n)]).reshape(m,n).astype(float)
    h=np.array([0 for i in range(m)]).reshape(n).astype(float)
    A=np.array([r.randint(0,9) for i in range(p*n)]).reshape(p,n).astype(float)
    b=np.array([r.randint(0,9) for i in range(p)]).reshape(p).astype(float)
    
    P_tch=torch.from_numpy(P)
    q_tch=torch.from_numpy(q)
    G_tch=torch.from_numpy(G)
    h_tch=torch.from_numpy(h)
    A_tch=torch.from_numpy(A)
    b_tch=torch.from_numpy(b)
    
    #get cvxpy solution
    P = sqrtm(P)
    start=time.time()
    x,y=qp_cvxpy(n,P,q,G,h,A,b)
    cvxpy_time+=time.time()-start
    re=pd.Series({"seed":s})
    for i in range(n):
      if y is not None:
        re=re.append(pd.Series({str(i)+"cp":np.round(y[i],6)}))
      else:
        re=re.append(pd.Series({str(i)+"cp":0}))
    
    #get mpc solution
    try:
      start=time.time()
      solution,_= mpc_solver.solve(P_tch.view(nbatch,n,n), q_tch.view(nbatch,n), G_tch.view(nbatch,m,n), h_tch.view(nbatch,m), A_tch.view(nbatch,p,n), b_tch.view(nbatch,p))
      # mpc_class.opt(P_tch.view(nbatch,n,n), q_tch.view(nbatch,n), G_tch.view(nbatch,m,n), h_tch.view(nbatch,m), A_tch.view(nbatch,p,n), b_tch.view(nbatch,p))
      mpc_time+=time.time()-start
      # print(solution.numpy().reshape(-1))
      mpc_solution=solution.numpy().reshape(-1)
    except:
        mpc_solution=[0]*n
    for i in range(n):
        re=re.append(pd.Series({str(i)+"mpc":np.round(mpc_solution[i],6)}))

    #get qpth solution
    # try:
    start=time.time()
    solution,_,_,_ = qpth_class.qpth_opt(P_tch.view(nbatch,n,n), q_tch.view(nbatch,n), G_tch.view(nbatch,m,n), h_tch.view(nbatch,m), A_tch.view(nbatch,p,n), b_tch.view(nbatch,p))
    qpth_time+=time.time()-start
      # print(solution.numpy().reshape(-1))
    qpth_solution=solution.detach().numpy()[0]
    # except:
        # qpth_solution=[0]*n
    for i in range(n):
        re=re.append(pd.Series({str(i)+"qpt":np.round(qpth_solution[i],6)}))

    results=results.append(re,ignore_index=True )

qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
no of mu not converged:  1
mpc warning: Residuals not converged, need more itrations
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an inaccurate and potentially incorrect solution.
qpth warning: Returning an 

In [17]:
print("cvxpy time: ", cvxpy_time)
print("mpc time: ", mpc_time)
print("qpth time: ", qpth_time)

cvxpy time:  101.97466611862183
mpc time:  190.90161299705505
qpth time:  233.41819763183594


In [18]:
error_qpt=np.zeros(len(results))
error_mpc=np.zeros(len(results))
for i in range(n):
    error_qpt+=results[str(i)+"cp"]-results[str(i)+"qpt"]
    error_mpc+=results[str(i)+"cp"]-results[str(i)+"mpc"]
results["cp-qpt-error"]=error_qpt   
results["cp-qpt-er_flag"]=error_qpt!=0

results["cp-mpc-error"]=error_mpc    
results["cp-mpc-er_flag"]=error_mpc!=0

#calculate optimum values and store them
cp_=torch.tensor(np.array(results[cols_cp])).view(len(df),nx,1)
cp_opt=np.round(((0.5*torch.bmm(torch.transpose(cp_,dim0=2,dim1=1),
                                torch.bmm(Q_batched,cp_)))+torch.bmm(p_batched.unsqueeze(-1).transpose(2,1),cp_)).view(-1).numpy(),6)
results["cp-opt"]=cp_opt
mpc=torch.tensor(np.array(results[cols_mpc])).view(len(df),nx,1)
mpc_opt=np.round(((0.5*torch.bmm(torch.transpose(mpc,dim0=2,dim1=1),
                                 torch.bmm(Q_batched.view(len(df),nx,nx),mpc)))+torch.bmm(p_batched.unsqueeze(-1).transpose(2,1),mpc)).view(-1).numpy(),6)
results["mpc-opt"]=mpc_opt
qpt=torch.tensor(np.array(results[cols_qpt])).view(len(df),nx,1)
qpt_opt=np.round(((0.5*torch.bmm(torch.transpose(qpt,dim0=2,dim1=1),
                                 torch.bmm(Q_batched.view(len(df),nx,nx),qpt)))+torch.bmm(p_batched.unsqueeze(-1).transpose(2,1),qpt)).view(-1).numpy(),6)
results["qpt-opt"]=qpt_opt


print("No of solved problems",len(seeds))
cvxpy_list=[]
for i in range(len(results)-1): #remove rows where cvxpy threw error
    if all(results[cols_cp].iloc[i,:]==0):
      cvxpy_list.append(i)
results=results.drop(cvxpy_list,axis=0)
print("No.of errors from MPC Method: ",len(results[results['cp-mpc-error']!=0])," (",round(len(results[results['cp-mpc-error']!=0])*100/len(results),2),"%)")
print("No.of errors from qpth: ",len(results[results['cp-qpt-error']!=0])," (",round(len(results[results['cp-qpt-error']!=0])*100/len(results),2),"%)")

cols_order=["seed","cp-opt","mpc-opt","qpt-opt","cp-mpc-error","cp-mpc-er_flag","cp-qpt-error","cp-qpt-er_flag"]+ cols_cp+cols_mpc+cols_qpt
results=results[cols_order]

No of solved problems 10000
No.of errors from MPC Method:  26  ( 0.26 %)
No.of errors from qpth:  438  ( 4.39 %)


In [19]:
results[results["cp-mpc-er_flag"]]

,seed,cp-opt,mpc-opt,qpt-opt,cp-mpc-error,cp-mpc-er_flag,cp-qpt-error,cp-qpt-er_flag,0cp,1cp,2cp,3cp,4cp,5cp,0mpc,1mpc,2mpc,3mpc,4mpc,5mpc,0qpt,1qpt,2qpt,3qpt,4qpt,5qpt
1303,3251.0,1079.911924,1075.866849,1.999779,5.868100e-02,True,30.284424,True,39.207787,-39.756540,9.266674,-24.515612,31.604835,14.898972,39.132071,-39.679418,9.249718,-24.468080,31.542957,14.870187,0.247299,-0.135418,0.108081,0.102240,-0.323591,0.423081
1499,3709.0,2953.490568,2953.490548,16.922572,-1.000000e-06,True,-9.442013,True,-20.992844,17.648964,-29.570810,23.216572,-0.870245,0.738983,-20.992844,17.648964,-29.570810,23.216573,-0.870245,0.738983,-1.950798,2.370733,-2.436584,2.239139,-0.075603,-0.534254
1563,3885.0,5.826862,5.910509,5.826862,1.790200e-02,True,0.000000,False,0.211054,0.100187,0.419184,0.417974,0.302179,-0.123173,0.230354,0.082182,0.396667,0.442659,0.324449,-0.166808,0.211054,0.100187,0.419184,0.417974,0.302179,-0.123173
1654,4085.0,104.411748,104.076363,104.076363,1.338100e-02,True,0.013381,True,-2.387754,1.054420,-0.800544,1.278809,9.136928,-3.278140,-2.381643,1.048309,-0.798309,1.277778,9.115942,-3.271739,-2.381643,1.048309,-0.798309,1.277778,9.115942,-3.271739
2090,5130.0,3693.790481,3658.645187,15.595324,6.028700e-02,True,12.177610,True,-20.239846,19.914573,-5.128942,22.540444,-9.099782,5.446464,-20.141635,19.815589,-5.098859,22.431559,-9.057034,5.423004,-0.336779,-0.241715,1.080021,0.297595,-0.320653,0.776832
2795,6815.0,128.800444,127.902069,2.492136,-1.239800e-02,True,-3.262399,True,-2.736962,0.879507,-3.153703,1.394740,-5.356858,6.943179,-2.725664,0.876106,-3.141593,1.389381,-5.336283,6.920354,0.250074,-0.067904,0.095795,-0.020598,0.111927,0.863008
2842,6916.0,11631.000912,11294.337377,32.768035,-1.895405e+00,True,-126.004612,True,-1.357289,-45.782718,43.433463,66.969304,-167.384097,-20.964545,-1.333333,-45.071429,42.785714,65.976190,-164.904762,-20.642857,0.186812,1.476416,0.402549,0.623395,-2.291597,0.521155
3477,8508.0,159.332401,160.008859,3.035600,-1.063500e-02,True,5.746063,True,-15.379678,5.795875,21.712877,7.981493,0.111556,-12.831594,-15.407703,5.805959,21.750727,7.994913,0.111919,-12.854651,-0.246588,0.318419,1.285706,0.803005,-0.137704,-0.378372
4809,11761.0,166050.666640,166051.104405,-33.582401,2.250000e-04,True,-170.514250,True,-485.072165,227.559093,-82.505642,-211.118501,407.049279,-28.501878,-485.072797,227.559387,-82.505747,-211.118774,407.049808,-28.501916,-4.887490,2.484790,-2.521724,-1.540369,6.229804,-1.840575
5275,12879.0,177623.126784,177624.312304,7.368795,-1.932000e-03,True,574.223678,True,685.634070,-193.201697,78.967478,-93.108194,195.459756,-96.084722,685.636364,-193.202346,78.967742,-93.108504,195.460411,-96.085044,2.795605,0.322971,-0.221731,-0.205320,-0.336795,1.088283


In [20]:
results[results["cp-qpt-er_flag"]]

,seed,cp-opt,mpc-opt,qpt-opt,cp-mpc-error,cp-mpc-er_flag,cp-qpt-error,cp-qpt-er_flag,0cp,1cp,2cp,3cp,4cp,5cp,0mpc,1mpc,2mpc,3mpc,4mpc,5mpc,0qpt,1qpt,2qpt,3qpt,4qpt,5qpt
4,17.0,784.641274,784.641274,34.013122,0.0,False,3.240155,True,-3.547789,-6.946498,9.317247,-10.633284,11.809383,6.198548,-3.547789,-6.946498,9.317247,-10.633284,11.809383,6.198548,-0.005024,-0.859741,2.038402,-2.559575,2.312269,2.031121
5,19.0,-76.299551,-76.299551,-37.616868,0.0,False,-4.666899,True,15.902075,-3.555210,-7.195557,-11.752652,4.153690,-6.635221,15.902075,-3.555210,-7.195557,-11.752652,4.153690,-6.635221,4.298103,0.976133,-2.610169,-4.747300,1.157036,-3.489779
9,28.0,270.015443,270.015443,4.717538,0.0,False,-11.580063,True,-1.204545,-4.892857,8.237013,12.048701,-4.370130,-20.438312,-1.204545,-4.892857,8.237013,12.048701,-4.370130,-20.438312,0.122440,0.414898,0.482550,-0.005486,0.588872,-0.643341
20,55.0,93.819403,93.819403,1.924895,0.0,False,2.297303,True,-2.181463,9.261561,5.237616,-9.060816,-3.226330,3.304756,-2.181463,9.261561,5.237616,-9.060816,-3.226330,3.304756,0.161856,1.463096,-0.071658,-1.454638,0.118772,0.820593
53,135.0,503.590906,503.590906,27.761086,0.0,False,7.421150,True,3.380764,19.510594,-17.821770,0.172206,-6.386165,10.014125,3.380764,19.510594,-17.821770,0.172206,-6.386165,10.014125,0.758886,2.954084,0.730931,-0.694544,-1.949890,-0.350863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,23861.0,22.990069,22.990069,23.910629,0.0,False,-0.002078,True,0.636109,-1.953110,0.737410,2.858210,-1.856395,0.794285,0.636109,-1.953110,0.737410,2.858210,-1.856395,0.794285,0.706113,-2.306586,0.613622,3.259270,-2.065296,1.011464
9806,23914.0,3183.889402,3183.889402,23.378345,0.0,False,-59.503841,True,17.171324,30.214338,19.702795,-63.574484,-16.045443,-44.008749,17.171324,30.214338,19.702795,-63.574484,-16.045443,-44.008749,3.214159,0.222939,0.277309,0.420487,-2.699560,1.528288
9860,24041.0,30.568261,30.568261,4.990234,0.0,False,-1.149437,True,-7.165289,1.827479,-0.976240,5.632231,-3.401860,4.363636,-7.165289,1.827479,-0.976240,5.632231,-3.401860,4.363636,-0.752498,0.591942,0.448412,0.955744,-0.327006,0.512800
9876,24076.0,57.323380,57.323380,17.758830,0.0,False,-2.263076,True,-2.320819,4.655290,-0.781570,-0.573379,4.054608,-5.907850,-2.320819,4.655290,-0.781570,-0.573379,4.054608,-5.907850,-0.037944,0.542821,-0.637945,1.055328,0.539516,-0.072420
